In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [4]:
# Define model name and number of labels
model_name = "google-bert/bert-base-uncased"
num_labels = 49  # Replace with the actual number of labels in your data

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
data = pd.read_excel('/content/manipulated_data.xlsx')

In [6]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(
    data,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 824
Number of rows in test set: 207


In [7]:
# Split the data into texts and labels
train_texts = train_df['Text'].tolist()
train_labels = train_df.iloc[:, 1:].values.tolist()

In [8]:
test_split = 0.5

# Initial train and test split.
val_df, test_df = train_test_split(
    test_df,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 103
Number of rows in test set: 104


In [9]:
# Split the data into texts and labels
val_texts = val_df['Text'].tolist()
val_labels = val_df.iloc[:, 1:].values.tolist()

In [10]:
#train_texts
#train_labels

In [11]:
# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)

In [12]:
# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [13]:
#train_encodings[0]
#train_encodings

In [14]:
from torch.utils.data import Dataset
'''
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item'''

'\nclass CustomDataset(Dataset):\n    def __init__(self, encodings, labels):\n        self.encodings = encodings\n        self.labels = labels\n    \n    def __len__(self):\n        return len(self.encodings["input_ids"])\n    \n    def __getitem__(self, idx):\n        item = {key: val[idx] for key, val in self.encodings.items()}\n        item["labels"] = self.labels[idx]\n        return item'

In [15]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.encodings["input_ids"][idx]),
            'attention_mask': torch.tensor(self.encodings["attention_mask"][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
        return item

In [16]:
# Create custom dataset
train_dataset = CustomDataset(train_encodings, train_labels)

# Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)

In [17]:
# Create custom dataset
val_dataset = CustomDataset(val_encodings, val_labels)

# Create dataloader
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [18]:
#next(iter(train_dataset))
#next(iter(train_dataloader))

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()  # Set the model to training mode
    train_loss = 0.0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/ {num_epochs}", leave=False):
        input_ids = batch['input_ids'].to(device)  # Move input to device
        attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
        labels = batch['labels'].to(device)  # Move labels to device

        optimizer.zero_grad()  # Clear gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels.float())
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        train_loss += loss.item()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Validation", leave=False):
            input_ids = batch['input_ids'].to(device)  # Move input to device
            attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
            labels = batch['labels'].to(device)  # Move labels to device

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels.float())

            val_loss += loss.item()

    # Calculate average loss
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

Epoch 1/10, Train Loss: 0.16517600574467367, Val Loss: 0.12117169052362442


Epoch 2/10, Train Loss: 0.12860092732295828, Val Loss: 0.12131014724190418


Epoch 3/10, Train Loss: 0.12924206368986843, Val Loss: 0.12140619955383815


Epoch 4/10, Train Loss: 0.12955770350413995, Val Loss: 0.12255943108063477


Epoch 5/10, Train Loss: 0.12948133536044834, Val Loss: 0.12251992609638435


Epoch 6/10, Train Loss: 0.1295399826417849, Val Loss: 0.12145414690558727


Epoch 7/10, Train Loss: 0.12883174039640474, Val Loss: 0.12158836424350739


Epoch 8/ 10:  12%|█▏        | 24/206 [01:41<12:44,  4.20s/it]

In [ ]:
test_texts =test_df['Text'].tolist()
test_labels = test_df.iloc[:, 1:].values.tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)
test_dataset = CustomDataset(test_encodings, val_labels)
test_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Put the model in evaluation mode
model.eval()

# Lists to store predictions and labels
all_predictions = []
all_labels = []

# Iterate through the test dataset
for batch in tqdm(test_dataloader, desc="Testing", leave=False):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # Disable gradient computation
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Apply sigmoid activation and round to obtain binary predictions
        predictions = torch.sigmoid(logits).cpu().numpy()
        rounded_predictions = (predictions > 0.5).astype(int)

        # Append predictions and labels to lists
        all_predictions.extend(rounded_predictions)
        all_labels.extend(labels.cpu().numpy())

# Convert lists to numpy arrays for easier manipulation
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
conf_matrix = confusion_matrix(all_labels.argmax(axis=1), all_predictions.argmax(axis=1))
class_report = classification_report(all_labels, all_predictions)

# Print evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)